In [ ]:
from ngsolve import *
import numpy as np
import scipy
import scipy.sparse as sparse
import math
import netgen.gui
#from ngsolve.webgui import Draw


class ReducedBasis:

    def __init__(self, fes, f):
        self.setSpace(fes, f)
        self.logging = True

In [ ]:
class ReducedBasis(ReducedBasis):

    def setSpace(self, fes, func):

        self.fes = fes
        
        # assume robin bnd if fes is complex
        if fes.is_complex:
            self.robin = True
        else:
            self.robin = False

        # set (bi-)linear forms and matrices
        self.omega = Parameter(0)
        
        ##########################################
        ## TODO: do this better
        u,v =self.fes.TnT()
        self.a = BilinearForm(self.fes)
        self.a += (grad(u)*grad(v) - self.omega*self.omega *u*v) * dx
        ## what happens if "robin" doesn't exist as boundary?
        if self.robin:
            if 'robin' in self.fes.mesh.GetBoundaries():
                self.a += -1j*self.omega*u*v*ds (definedon=self.fes.mesh.Boundaries("robin"))
            else:
                self.a += -1j*self.omega*u*v*ds
        self.a.Assemble()
        ##########################################
        
        # store ainv for better performance
        self.ainv = self.a.mat.Inverse(self.fes.FreeDofs(), inverse="sparsecholesky")


        self.f = LinearForm(self.fes)
        self.f += func * v * dx
        self.f.Assemble()

        self.k_blf = BilinearForm(self.fes)
        self.k_blf += grad(u)*grad(v)*dx
        self.k_blf.Assemble()
        rows,cols,vals = self.k_blf.mat.COO()
        self.K_orig = sparse.csr_matrix((vals,(rows,cols)))

        self.m_blf = BilinearForm(self.fes)
        self.m_blf += u*v * dx
        self.m_blf.Assemble()
        rows,cols,vals = self.m_blf.mat.COO()
        self.M_orig = sparse.csr_matrix((vals,(rows,cols)))

        if self.robin:
            self.r_blf = BilinearForm(self.fes)
            if 'robin' in self.fes.mesh.GetBoundaries():
                self.r_blf += -1j*u*v*ds (definedon=self.fes.mesh.Boundaries("robin"))
            else:
                self.r_blf += -1j*u*v*ds
    
            self.r_blf.Assemble()
            rows,cols,vals = self.r_blf.mat.COO()
            self.R_orig = sparse.csr_matrix((vals,(rows,cols)))
    

        # initialize grid functions
        self.gfu = GridFunction(self.fes)
        self.drawu = GridFunction(self.fes)

        self.reset()

    
    def reset(self):
        # reset dynamically updated parameters
        self.K_red = None
        self.M_red = None
        self.P_red = None
        self.R_red = None
        self.sol = []
        self.norm = [] 
        self.residual = []
        self.__V = None  # snapshot solutions
        self.__snapshots_updated = True
        self.__snapshots = [] # snapshot parameters
        self.__indices = []


In [ ]:
class ReducedBasis(ReducedBasis):

    def setSnapshots(self, new_snapshots, reset = True, compute_RB = True):
        
        # TODO: check that snapshots do not already exist
        if len(self.__snapshots) > 0 and not reset:
            if self.logging: print("append snapshots with {}".format(new_snapshots))
            self.__snapshots = np.append(self.__snapshots, np.array(new_snapshots))
        else:
            if self.logging: print("set snapshots and reset basis")
            self.__snapshots = np.array(new_snapshots)
            self.__V = None

        self.__snapshots_updated = True

        # store smallest and biggest snapshot parameter
        self.omega_min = min(self.__snapshots)
        self.omega_max = max(self.__snapshots)

        # store indices of snapshots in ascending order
        tmp = self.__snapshots
        zip_to_sort = list(zip(tmp, range(len(tmp))))
        sorted_zip = sorted(zip_to_sort, key=lambda x: x[0], reverse=False)
        self.__indices = [tup[1] for tup in sorted_zip]

        if compute_RB:
            self.__computeRB()

    def getSnapshots(self):
        return self.__snapshots[self.__indices]

In [ ]:
class ReducedBasis(ReducedBasis):
    
    def __computeRB(self):


        if self.logging: print("compute Reduced Basis")

        if len(self.__snapshots) == 0:
            if self.logging: print(""" no snapshots given, please call 'instance.setSnapshots' first""")
            return
        
        dim_orig = len(self.gfu.vec)
        dim_red = len(self.__snapshots)

        if self.robin:
            V_tmp = np.zeros((dim_orig, dim_red), dtype=complex )
        else:
            V_tmp = np.zeros((dim_orig, dim_red))

        # extend basis if it already exists
        try: 
            existing_basis_len = self.__V.shape[1]
            V_tmp[:,0:existing_basis_len] = self.__V
            if self.logging: print("extend reduced basis")

        except:
            existing_basis_len = 0


        with TaskManager():

            for n in range(0+existing_basis_len, dim_red):
                _omega = self.__snapshots[n]
                
                # compute FEM solution for parameter _omega
                self.omega.Set(_omega)
                self.a.Assemble()

                self.ainv.Update()
                self.gfu.vec.data = self.ainv * self.f.vec

                V_tmp[:,n] = self.gfu.vec.FV().NumPy()        


            # do QR-decomposition 
            q, r = np.linalg.qr(V_tmp)
            self.__V = V_tmp.dot(np.linalg.inv(r))

            # rearange V and snapshots due to the order of the snapshots
            self.__V = self.__V[:, self.__indices]
            self.__snapshots = self.__snapshots[self.__indices]
            self.__indices = range(dim_red)

            # set system in reduced basis space
            self.K_red = np.transpose(self.__V).dot(self.K_orig.dot(self.__V))
            self.M_red = np.transpose(self.__V).dot(self.M_orig.dot(self.__V))
            self.P_red = np.transpose(self.__V).dot(self.f.vec.data)

            if self.robin:
                self.R_red = np.transpose(self.__V).dot(self.R_orig.dot(self.__V))

            self.__snapshots_updated = False
            if self.logging: print("finished computing Reduced Basis")


In [ ]:
class ReducedBasis(ReducedBasis):
    
    def draw(self, omega, redraw=False):
        
        if self.__snapshots_updated:
            self.__computeRB()
        
        # compute reduced basis
        
        ## TODO: make LU decomposition once  (for M and K) and use np.linalg.solve_triangular
        ## (vorwaerts- und rueckwaertssubstitution)
        if self.robin:
            Ainv = np.linalg.inv(self.K_red-omega*omega*self.M_red+omega*self.R_red)
        else:
            Ainv = np.linalg.inv(self.K_red-omega*omega*self.M_red)
        red_sol_vec = np.matmul(Ainv, self.P_red)
        
        
        self.drawu.vec.FV().NumPy()[:] = self.__V.dot(red_sol_vec)[:]
        if self.logging: print("omega: {}, norm of solution: {}".format(omega, Integrate ( (self.drawu)*(self.drawu), self.fes.mesh)))
       
        # draw solution
        if not redraw:
            self.scene = Draw(self.drawu)
        else:
            self.scene.Redraw()

In [ ]:
from netgen.geom2d import *
import time

geo = SplineGeometry()

r1 = 1.5
r2 = 0.35
h1 = 7
h2 = 1.5
h3 = 0.5
d = 0.2


x_val = [0, 2*r1, 2*r1, r1+r2, r1+r2, r1+r2-d, r1+r2-d, 2*r1-d, 2*r1-d, d, d, r1-r2+d, r1-r2+d, r1-r2, r1-r2, 0]
y_val = [0, 0, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1, d, d, h1, h1+h2, h1+h2+h3, h1+h2+h3, h1+h2, h1]

assert len(x_val)==len(y_val), "lengths of x and y values must be the same"
points = [None]*len(x_val)
amount_points = len(points)
for j in range(0,amount_points):
    points[j] = geo.AppendPoint(x_val[j], y_val[j])

for j in range(0,amount_points):
    geo.Append(["line", points[j], points[(j+1) % amount_points]], leftdomain=0, rightdomain=1, bc = "bottle")

geo.AddRectangle((-1,-1),(2*r1+1, h1+h2+h3+1),bc="robin")
geo.SetMaterial (1, "air")


mesh = Mesh(geo.GenerateMesh(maxh=0.2))

fes = H1(mesh, order=5, complex=True, definedon="air")#, dirichlet="bottle")

## peak on top of bottle
func1 = 1e5*exp(-(50**2)*((x-r1)**2 + (y-(h1+h2+h3+0.3))**2))

## peak left from bottle
func2 = 1e5*exp(-(50**2)*((x-r2)**2 + (y-(h1+h2+h3+0.3))**2))

## peak right from bottle
func3 = 1e5*exp(-(50**2)*((x-(r2+r1))**2 + (y-(h1+h2+h3+0.3))**2))

## peak in the bottle
func4 = 1e5*exp(-(50**2)*((x-(r1))**2 + (y-(h1+0.5))**2))

## peak in neck
func5 = 1e5*exp(-(50**2)*((x-r1)**2 + (y-(h1+h2+h3/2))**2))


space = ReducedBasis(fes, func1)
snapshots = np.arange(0,30, 2)
space.setSnapshots(snapshots)

slider_func = lambda x: space.draw(x, redraw=True)
space.draw(space.omega_min)

In [ ]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

interact(slider_func, x=widgets.FloatSlider(min=space.omega_min, max=space.omega_max, step=0.1, value=space.omega_min))